In [1]:
!pip install langchain langchain-community huggingface_hub unstructured pdfminer.six
#!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py
#!pip install pillow_heif pdf2image
!pip install sentence-transformers
!pip install faiss-gpu
!pip install pypdf

import os
import pandas as pd
import numpy as np
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFaceEndpoint
from tqdm import tqdm

os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

In [2]:
text = """
1. Вступление
Описание курса, начальные требования, для кого этот курс.
1.1 Как правильно входить в курс
Цели курса, канал и комьюнити, глоссарий, PET-проект, хард режим vs лайт режим?
1.2 Общий подход и точки улучшения приложений с LLM
Рассказываем почему важно разбираться в LLM. Нужен ли ИИ обычному человеку?
1.3 API ключ курса или от OpenAI?
1.3.1 Ключ от команды курса
Получаем ключ в боте.
1.3.2 Ключ от OpenAI
Получаем официальный ключ от OpenAI.
1.3.3 Ключ от HuggingFace
Ныряем в Open Source и получаем ключ от HuggingFace.
2. Промптинг - объясни LLM, что тебе от неё надо!
2.1 Введение в Prompt Engineering
Поясняем за промпты. Техники и лайфхаки для промптинга. Из чего состоит промпт?
2.2 Дизайн промптов в LangChain
Few-shot learning. Output Parser.
2.2.1 Введение в LangChain
Рассказываем про преимущества LangChain.
3. LangChain или причем тут попугаи?
3.1 Память в LangChain
Переводим LLM в чат-режим. Типы памяти.
3.2 Chains - собери свою цепь
Chains & LCEL.
3.3 Агенты intro
Агенты и цепи. Инструменты (tools).
"""

In [3]:
text.split('\n')

['',
 '1. Вступление',
 'Описание курса, начальные требования, для кого этот курс.',
 '1.1 Как правильно входить в курс',
 'Цели курса, канал и комьюнити, глоссарий, PET-проект, хард режим vs лайт режим?',
 '1.2 Общий подход и точки улучшения приложений с LLM',
 'Рассказываем почему важно разбираться в LLM. Нужен ли ИИ обычному человеку?',
 '1.3 API ключ курса или от OpenAI?',
 '1.3.1 Ключ от команды курса',
 'Получаем ключ в боте.',
 '1.3.2 Ключ от OpenAI',
 'Получаем официальный ключ от OpenAI.',
 '1.3.3 Ключ от HuggingFace',
 'Ныряем в Open Source и получаем ключ от HuggingFace.',
 '2. Промптинг - объясни LLM, что тебе от неё надо!',
 '2.1 Введение в Prompt Engineering',
 'Поясняем за промпты. Техники и лайфхаки для промптинга. Из чего состоит промпт?',
 '2.2 Дизайн промптов в LangChain',
 'Few-shot learning. Output Parser.',
 '2.2.1 Введение в LangChain',
 'Рассказываем про преимущества LangChain.',
 '3. LangChain или причем тут попугаи?',
 '3.1 Память в LangChain',
 'Переводим LLM

In [4]:
def get_content_from_sections(text: str): # -> List[str]:
    out = []
    box = []
    headers = [str(i)+'.' for i in range(1, 100)]
    len_text = len(text.split('\n'))
    for index, i in enumerate(text.split('\n'), 1):
      if i.strip():
        if (box and i.split()[0] in headers):
          out.append(" ".join(box))
          box = []

        if not i.split()[0].replace('.', '').isdigit():
          box.append(i)

      if (index == len_text) and box:
        out.append(" ".join(box))
        box = []
    return out

In [5]:
get_content_from_sections(text)

['Описание курса, начальные требования, для кого этот курс. Цели курса, канал и комьюнити, глоссарий, PET-проект, хард режим vs лайт режим? Рассказываем почему важно разбираться в LLM. Нужен ли ИИ обычному человеку? Получаем ключ в боте. Получаем официальный ключ от OpenAI. Ныряем в Open Source и получаем ключ от HuggingFace.',
 'Поясняем за промпты. Техники и лайфхаки для промптинга. Из чего состоит промпт? Few-shot learning. Output Parser. Рассказываем про преимущества LangChain.',
 'Переводим LLM в чат-режим. Типы памяти. Chains & LCEL. Агенты и цепи. Инструменты (tools).']

# RAG

In [6]:
df = pd.read_csv('/content/pushkin_questions.csv')

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/The_Daughter_of_The_Commandant.pdf")
docs = loader.load_and_split()

In [8]:
loader

In [9]:
docs[1].page_content

'www.a4format.ru   \n 2\nнелестно  отзывается  о дочери  коменданта  Миронова  Маше . Швабрин  и Гринев  при-\nглашены  на обед  в семью  коменданта . По пути  Гринев  видит  «учения »: комендант  Иван  \nКузьмич  Миронов  командует  взводом  инвалидов . Сам он при этом  одет  «в колпаке  \nи китайчатом  халате ». \nГлава  4 \nПоединок  \nГринев  очень  привязывается  к семейству  коменданта . Его производят  в офицеры . \nГринев  много  общается  со Швабриным , но тот ему нравится  все меньше , а особенно  его \nколкие  замечания  о Маше . Гринев  посвящает  Маше  посредственные  любовные  стихи . \nШвабрин  резко  их критикует , в разговоре  с Гриневым  оскорбляет  Машу . Гринев  называет  \nего лжецом , Швабрин  требует  сатисфакции . Перед  дуэлью  по приказанию  Василисы  \nЕгоровны  их арестовывают , дворовая  девка  Палашка  даже  отнимает  у них шпаги . Через  \nнекоторое  время  Гринев  узнает  от Маши , что Швабрин  за нее сватался , а она отказала  (этим  \nи объясняется  уп

In [10]:
# SPLITTING
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
'''
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
)
'''
splitter = CharacterTextSplitter(
    separator="Глава",  # символ-разделитель, по умолчанию переход к новому абзацу '\n\n'
    chunk_size=500,  # размер документа в символах
    chunk_overlap=100,  # насколько соседние документы могут перекрывать друг-друга
    length_function=len,  # функция, по которой считается размер документа
    is_separator_regex=False,  # является ли разделитель регулярным выражением
)


split_documents = splitter.create_documents([doc.page_content for doc in docs])
len(split_documents)

18

In [11]:
split_documents[0]

Document(page_content='www.a4format.ru   \nВсе произведения  школьной  программы  в кратком  изложении . — М.: АСТ , 1996.  \n \nИ.О. Родин , Т.М. Пименова  \nКапитанская  дочка  \nКраткое  содержание  \nГлава  1 \nСержант  гвардии')

In [12]:
#EMBEDDING
from langchain.embeddings import HuggingFaceEmbeddings

# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cpu"}
)

#embeddings = hf_embeddings_model.embed_documents(
#    [doc.page_content for doc in split_documents]
#)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
#VECTOR STORE
from langchain.vectorstores import FAISS

db = FAISS.from_documents(
    split_documents, hf_embeddings_model
)
db.save_local("faiss_db")  # можно сохранить базу локально, указав путь

In [14]:
df['question'].iloc[1]

'Где Маша остановилась в Петербурге?'

In [15]:
# База готова, теперь можно делать к ней запросы
db.similarity_search(df['question'].iloc[1])

[Document(page_content='14 \nСуд \nПо обвинению  Швабрина  Гринева  подозревают  в том, что он служил  Пугачеву . \nГринева  приговаривают  к ссылке  в Сибирь . Родители  Гринева  очень  привязались  к Маше . \nНе желая  злоупотреблять  их великодушием , Маша  едет в Петербург , останавливается  \nв Царском  Селе , в саду  встречает  императрицу  и просит  о милости  к Гриневу , объясняя , \nчто он попал  к Пугачеву  из-за нее. На аудиенции  императрица  обещает  устроить  судьбу  \nМаши  и простить  Гринева . Гринева  освобождают  из-под стражи . «Он присутствовал  при \nказни  Пугачева , который  узнал  его в толпе  и кивнул  ему головой , которая  через  минуту , \nмертвая  и окровавленная , показана  была  народу ».'),
 Document(page_content='11 \nМятежная  слобода  \nГринев  и Савельич  одни  отправляются  на помощь  Маше . По дороге  их хватают  \nлюди  Пугачева . Пугачев  допрашивает  Гринева  о его намерениях  в присутствии  наперс -\nников . «Один  из них, тщедушный  и сгорбле

In [16]:
#RETRIEVER

# Самый частый кейс - использование векторного хранилища и его методов для получения документов
retriever = db.as_retriever(
    search_type="similarity",  # тип поиска похожих документов
    k=2,  # количество возвращаемых документов (Default: 4)
    score_threshold=1.6,  # минимальный порог для поиска "similarity_score_threshold"
)
retriever.get_relevant_documents(
    df['question'].iloc[1]
)[0]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Document(page_content='14 \nСуд \nПо обвинению  Швабрина  Гринева  подозревают  в том, что он служил  Пугачеву . \nГринева  приговаривают  к ссылке  в Сибирь . Родители  Гринева  очень  привязались  к Маше . \nНе желая  злоупотреблять  их великодушием , Маша  едет в Петербург , останавливается  \nв Царском  Селе , в саду  встречает  императрицу  и просит  о милости  к Гриневу , объясняя , \nчто он попал  к Пугачеву  из-за нее. На аудиенции  императрица  обещает  устроить  судьбу  \nМаши  и простить  Гринева . Гринева  освобождают  из-под стражи . «Он присутствовал  при \nказни  Пугачева , который  узнал  его в толпе  и кивнул  ему головой , которая  через  минуту , \nмертвая  и окровавленная , показана  была  народу ».')

In [17]:
# FINAL PROMPT
llm = HuggingFaceEndpoint(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1")
#llm_chain = LLMChain(prompt=prompt, llm=llm)

# Создаём простой шаблон
template = """
Answer the question based only on the following context:

{context}

Question: {question}

Answer only in russian.
"""
# Создаём промпт из шаблона
#prompt = PromptTemplate(template=template, input_variables=["context"])
prompt = ChatPromptTemplate.from_template(template)


# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# Задаём ретривер
#retriever = db.as_retriever()


# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [18]:
df['question'].iloc[1]

'Где Маша остановилась в Петербурге?'

In [19]:

chain.invoke(df['question'].iloc[1])

'Маша остановилась в Царском Селе, в саду встречает императрицу.'

In [20]:
df['answer'] = ''
for i in tqdm(range(len(df))):
  res = chain.invoke(df['question'].iloc[i])
  res = res.replace('Answer:', '').replace('answer:', '').strip()

  if len(res) < 70:
    df['answer'].iloc[i] = res
  else:
    df['answer'].iloc[i] = res[:69]

100%|██████████| 15/15 [00:26<00:00,  1.74s/it]


In [21]:
df.to_csv('res.csv', index=False)

In [22]:
df

,question,answer
0,Кого убили раздетой на крыльце?,Василису Егоровну убили раздетой на крыльце.
1,Где Маша остановилась в Петербурге?,"Маша остановилась в Царском Селе, в саду встре..."
2,Почему Гринев и Савельич сбились с дороги?,Ответ: Гринев и Савельич сбились с дороги из-з...
3,Кто издевался над Машей и морил ее голодом?,Швабрин издевался над Машей и морил ее голодом.
4,Куда приговорили отправить Гринева в ссылку?,Отправили в ссылку в Сибирь.
5,Сколько детей было в семье Петра Гринева?,Ответ: В семье Петра Гринева было девять детей...
6,Кто воспитывал Петра Гринева?,Дядька Савельич воспитывал Петра Гринева.
7,"Сколько казаков попросил Гринев, чтобы очистит...","Гринев попросил у генерала полсотни казаков, ч..."
8,Как далеко находилась от города Белогорская кр...,Ответ: В тексте нет данных о расстоянии между ...
9,Как звали дочку коменданта?,Ответ: Маша
